In [1]:
from theFuncs import *

In [2]:
# List of stocks (top 100 in S&P 500) and get returns
# returns of S&P500 (overall) -- use index returns to get return of market
# Time horizon: Last 2 years
# Get rolling betas (3 months)

In [3]:
# load_DTB3_SPY().to_csv('./data/spy_dtb3.csv')

In [4]:
sp100 = os.listdir(os.getcwd()+'/data/stock_dfs')
sp100_tickers = []
for csvfile in sp100:
    sp100_tickers.append(csvfile[:-4])

# sp100_tickers

In [5]:
get_ticker_data_multisource(sp100_tickers[:6])

high         low        open       close      volume  \
ticker date                                                                     
BA     2019-01-02  323.950012  313.709991  316.190002  323.809998   3292200.0   
       2019-01-03  319.739990  309.399994  319.489990  310.899994   5705600.0   
       2019-01-04  328.440002  316.079987  316.690002  327.079987   4448800.0   
       2019-01-07  330.690002  323.350006  330.519989  328.109985   4030300.0   
       2019-01-08  341.250000  332.649994  334.109985  340.529999   4736500.0   
...                       ...         ...         ...         ...         ...   
V      2021-10-26  236.960007  231.330002  235.529999  231.820007   7814500.0   
       2021-10-27  224.750000  215.660004  224.750000  215.779999  22958100.0   
       2021-10-28  219.990005  209.009995  219.250000  209.839996  23199400.0   
       2021-10-29  213.669998  208.539993  209.210007  211.770004  14329800.0   
       2021-11-01  215.250000  211.449997  213.490005  212.460007  13210300.0   

                    spy_close  vix_close    DTB3  
ticker date                                       
BA     2019-01-02  238.694534  23.219999  0.0237  
       2019-01-03  232.998627  25.450001  0.0236  
       2019-01-04  240.803085  21.379999  0.0237  
       2019-01-07  242.701721  21.400000  0.0241  
       2019-01-08  244.981979  20.469999  0.0241  
...                       ...        ...     ...  
V      2021-10-26  455.959991  15.980000  0.0006  
       2021-10-27  453.940002  16.980000  0.0006  
       2021-10-28  458.320007  16.530001  0.0006  
       2021-10-29  459.250000  16.260000  0.0005  
       2021-11-01  460.040009  16.410000  0.0005  

[4290 rows x 8 columns]

In [6]:
model = RollingOLS.from_formula(
    "(RET - RF) ~ VWRETD", 
    data = final_returns, 
    window = 66
).fit(params_only=True)

final_returns.BETAS = model.params.VWRETD

final_returns.BETAS.tail()

NameError: name 'final_returns' is not defined

<a style='text-decoration:none;line-height:16px;display:flex;color:#5B5B62;padding:10px;justify-content:end;' href='https://deepnote.com?utm_source=created-in-deepnote-cell&projectId=80cc924b-f00e-4408-bf29-ac16ba2efaff' target="_blank">
 </img>
Created in <span style='font-weight:600;margin-left:4px;'>Deepnote</span></a>